In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, Row
from pyspark.sql.functions import *

In [3]:
spark = SparkContext(master='local', appName='DataFrames')
sqlContext = SQLContext(spark)

22/10/01 18:55:09 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.4 instead (on interface eno2)
22/10/01 18:55:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/01 18:55:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
path_file = '/home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/'

In [5]:
def eliminar_encabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [6]:
eventoSchemaRDD = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('deporte_id', IntegerType(), False)
])
eventoDF = sqlContext.read.schema(eventoSchemaRDD).option('header', 'True').csv(path_file + 'evento.csv')
eventoDF.show(5)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



22/10/01 18:55:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/evento.csv


In [7]:
paises_schema = StructType([
    StructField('id_equipo', IntegerType(), False),
    StructField('equipo', StringType(), False),
    StructField('sigla', StringType(), False)
])
paisesDF = sqlContext.read.schema(paises_schema).option('header', 'True').csv(path_file + 'paises.csv')
paisesDF.show(5)

+---------+--------------------+-----+
|id_equipo|              equipo|sigla|
+---------+--------------------+-----+
|        1|         30. Februar|  AUT|
|        2|A North American ...|  MEX|
|        3|           Acipactli|  MEX|
|        4|             Acturus|  ARG|
|        5|         Afghanistan|  AFG|
+---------+--------------------+-----+
only showing top 5 rows



22/10/01 18:55:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, equipo, sigla
 Schema: id_equipo, equipo, sigla
Expected: id_equipo but found: id
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/paises.csv


In [11]:
deportesOlimpicosRDDSchema = StructType(
    [StructField("deporte_id",IntegerType(),False),
     StructField("deporte",StringType(),False)
     ])

deportesDF = sqlContext.read.schema(deportesOlimpicosRDDSchema).option("header","true").csv(path_file+"deporte.csv")
deportesDF.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



In [9]:
deportistaOlimpicoRDD = spark.textFile(path_file+'deportista.csv') \
                        .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = spark.textFile(path_file+'deportista2.csv') \
                        .map(lambda l: l.split(","))
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminar_encabezado)
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    str(l[1]),
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))

deportistaSchema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False)
])

deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, deportistaSchema)
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [12]:
resultadoSchemaRDD = StructType([
    StructField('resultado_id', IntegerType(), False),
    StructField('medalla', StringType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('juego_id', IntegerType(), False),
    StructField('evento_id', IntegerType(), False)
])

resultadoDF = sqlContext.read.schema(resultadoSchemaRDD).option("header","true").csv(path_file+"resultados.csv")
resultadoDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [13]:
juegoSchema = StructType([
    StructField('juego_id', IntegerType(), False),
    StructField('nombre_juego', StringType(), False),
    StructField('anio', StringType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False)
])

juegoDataFrame = sqlContext.read.schema(juegoSchema).option("header","true").csv(path_file+"juegos.csv")
juegoDataFrame.show(5)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego|anio|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



22/10/01 19:00:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego, annio, temporada, ciudad
 Schema: juego_id, nombre_juego, anio, temporada, ciudad
Expected: juego_id but found: 
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/juegos.csv


In [14]:
deportistaDF.join(
    resultadoDF,
    deportistaDF.deportista_id == resultadoDF.deportista_id,
    'left'
) \
.join(
    juegoDataFrame,
    resultadoDF.juego_id == juegoDataFrame.juego_id,
    'left'
) \
.join(
    eventoDF,
    resultadoDF.evento_id == eventoDF.evento_id,
    'left'
) \
.select(deportistaDF.nombre,
        col('edad').alias('Edad Al Jugar'),
        'medalla',
        col('anio').alias('Año de juego'),
        eventoDF.nombre.alias('Nombre de disciplina')) \
.show(5)

22/10/01 19:01:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , annio
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/juegos.csv
22/10/01 19:01:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento
 Schema: evento_id, nombre
Expected: nombre but found: evento
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/evento.csv


+--------------------+-------------+-------+------------+--------------------+
|              nombre|Edad Al Jugar|medalla|Año de juego|Nombre de disciplina|
+--------------------+-------------+-------+------------+--------------------+
|           A Dijiang|           24|     NA|        1992|Basketball Men's ...|
|            A Lamusi|           23|     NA|        2012|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|           24|     NA|        1920|Football Men's Fo...|
|Edgar Lindenau Aabye|           34|   Gold|        1900|Tug-Of-War Men's ...|
|Christine Jacoba ...|           21|     NA|        1994|Speed Skating Wom...|
+--------------------+-------------+-------+------------+--------------------+
only showing top 5 rows



In [29]:
resultadoDF.filter(resultadoDF.medalla != 'NA') \
    .join(
        deportistaDF,
        resultadoDF.deportista_id == deportistaDF.deportista_id,
        'left') \
    .join(
        paisesDF,
        deportistaDF.equipo_id == paisesDF.id_equipo,
        'left') \
    .select(resultadoDF.medalla, paisesDF.equipo, paisesDF.sigla) \
    .sort(col('sigla').desc()) \
    .show()

22/10/01 17:17:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, equipo, sigla
 Schema: id_equipo, equipo, sigla
Expected: id_equipo but found: id
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/paises.csv


+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [30]:
medallistaXAnio = deportistaDF.join(resultadoDF, deportistaDF.deportista_id ==resultadoDF.deportista_id,"left") \
        .join(juegoDataFrame, juegoDataFrame.juego_id == resultadoDF.juego_id,"left") \
        .join(paisesDF,deportistaDF.equipo_id == paisesDF.id_equipo,"left") \
        .join(eventoDF, eventoDF.evento_id == resultadoDF.evento_id,"left") \
        .join(deportesDF,eventoDF.deporte_id == deportesDF.deporte_id,"left") \
        .select("sigla",
                "anio",
                "medalla",
                eventoDF.nombre.alias("Nombre subdisciplina"),
                deportesDF.deporte.alias("Nombre disciplina"),
                deportistaDF.nombre)
medallistaXAnio.show(5)

22/10/01 17:17:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , annio
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/juegos.csv
22/10/01 17:17:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, sigla
 Schema: id_equipo, sigla
Expected: id_equipo but found: id
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/paises.csv
22/10/01 17:17:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/evento.csv


+-----+----+-------+--------------------+-----------------+--------------------+
|sigla|anio|medalla|Nombre subdisciplina|Nombre disciplina|              nombre|
+-----+----+-------+--------------------+-----------------+--------------------+
|  CHN|1992|     NA|Basketball Men's ...|       Basketball|           A Dijiang|
|  CHN|2012|     NA|Judo Men's Extra-...|             Judo|            A Lamusi|
|  DEN|1920|     NA|Football Men's Fo...|         Football| Gunnar Nielsen Aaby|
|  SWE|1900|   Gold|Tug-Of-War Men's ...|       Tug-Of-War|Edgar Lindenau Aabye|
|  NED|1994|     NA|Speed Skating Wom...|    Speed Skating|Christine Jacoba ...|
+-----+----+-------+--------------------+-----------------+--------------------+
only showing top 5 rows



In [31]:
medallistaXAnio2 = medallistaXAnio.filter(medallistaXAnio.medalla != 'NA') \
    .groupBy('sigla', 'anio', 'Nombre subdisciplina') \
    .count().sort('anio')
medallistaXAnio2.show()

22/10/01 17:17:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , annio
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/juegos.csv
22/10/01 17:17:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/evento.csv
22/10/01 17:17:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, sigla
 Schema: id_equipo, sigla
Expected: id_equipo but found: id
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/paises.csv


+-----+----+--------------------+-----+
|sigla|anio|Nombre subdisciplina|count|
+-----+----+--------------------+-----+
|  AUT|1896|Cycling Men's 10,...|    1|
|  FRA|1896|Cycling Men's 100...|    1|
|  AUT|1896|Cycling Men's 12-...|    1|
|  GER|1896|Gymnastics Men's ...|    1|
|  GRE|1896|Cycling Men's 333...|    1|
|  HUN|1896|Tennis Men's Singles|    1|
|  GER|1896|Gymnastics Men's ...|   10|
|  GER|1896|Wrestling Men's U...|    1|
|  FRA|1896|Cycling Men's 10,...|    2|
|  USA|1896|Athletics Men's D...|    1|
|  GRE|1896|Gymnastics Men's ...|    2|
|  GER|1896|Gymnastics Men's ...|    1|
|  FRA|1896|Cycling Men's Sprint|    2|
|  GRE|1896|Athletics Men's M...|    2|
|  GER|1896|Gymnastics Men's ...|    2|
|  SUI|1896|Gymnastics Men's ...|    1|
|  GBR|1896|Athletics Men's 4...|    1|
|  USA|1896|Athletics Men's 4...|    2|
|  GRE|1896|Cycling Men's 100...|    1|
|  GER|1896|Gymnastics Men's ...|    2|
+-----+----+--------------------+-----+
only showing top 20 rows



In [32]:
medallistaXAnio2.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- anio: string (nullable = true)
 |-- Nombre subdisciplina: string (nullable = true)
 |-- count: long (nullable = false)



In [33]:
medallistaXAnio2.groupBy('sigla', 'anio') \
    .agg(sum('count').alias('Total de medallas'),
         avg('count').alias('Medallas promedio')).show()

22/10/01 17:17:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, sigla
 Schema: id_equipo, sigla
Expected: id_equipo but found: id
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/paises.csv
22/10/01 17:17:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , annio
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/juegos.csv
22/10/01 17:17:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: evento_id, evento, deporte_id
 Schema: evento_id, nombre, deporte_id
Expected: nombre but found: evento
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/evento.csv


+-----+----+-----------------+------------------+
|sigla|anio|Total de medallas| Medallas promedio|
+-----+----+-----------------+------------------+
|  FRG|1984|              162|2.7457627118644066|
|  RUS|1996|               75|1.7045454545454546|
|  HUN|1952|              104|3.1515151515151514|
|  AUS|1972|               20|1.1764705882352942|
|  GBR|1906|               42|               2.1|
|  PRK|2012|                6|               1.0|
|  BEL|2004|                3|               1.0|
|  DEN|1988|                7|              1.75|
|  LUX|1920|                1|               1.0|
|  CIV|2016|                2|               1.0|
|  BOT|2012|                1|               1.0|
|  NZL|1960|                3|               1.0|
|  GER|1968|               42|1.6153846153846154|
|  EGY|1936|                5|              1.25|
|  DEN|1948|               51|2.3181818181818183|
|  HUN|1906|               13|               1.3|
|  SUI|1924|               69|              2.76|


#### Persistencia y particionado

In [35]:
from pyspark.storagelevel import StorageLevel

In [36]:
medallistaXAnio.is_cached

False

In [37]:
medallistaXAnio.rdd.cache()

MapPartitionsRDD[165] at javaToPython at NativeMethodAccessorImpl.java:0

In [39]:
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [40]:
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[165] at javaToPython at NativeMethodAccessorImpl.java:0

In [41]:
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_2)

MapPartitionsRDD[165] at javaToPython at NativeMethodAccessorImpl.java:0

##### Crear StorageLevel

In [42]:
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True, True, False, False, 3)

In [43]:
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[165] at javaToPython at NativeMethodAccessorImpl.java:0

In [44]:
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[165] at javaToPython at NativeMethodAccessorImpl.java:0

In [34]:
#spark.stop()